# Data preparation for the UN SDG Indicators

**We use UN SDG's data set and convert this data set, so every country, continent, etc. is in a separate <code>csv</code> file.**

To get started, we download the entire available data from https://unstats.un.org/sdgs/indicators/database/ and call it <code>un_data.csv</code>.


Let's load the data set and look at its columns and rows to figure out how it is structured.


**We aim to have one pandas data frame per country, with all indicators. We save them as separate <code>csv</code> files.** 

Let's start with the usual imports and loading the data set.

In [1]:
import numpy as np
import pandas as pd
import math
import os

In [3]:
# loading data set
all_data = pd.read_csv('un_data.csv', dtype=object)
all_data.tail()

,Goal,Target,Indicator,SeriesCode,SeriesDescription,GeoAreaCode,GeoAreaName,TimePeriod,Value,Time_Detail,...,[Name of non-communicable disease],[Quantile],[Reporting Type],[Sex],[Tariff regime (status)],[Type of mobile technology],[Type of occupation],[Type of product],[Type of skill],[Type of speed]
1083970,17,17.9,17.9.1,DC_FTA_TOTAL,Total official development assistance (gross d...,894,Zambia,2012,75.97718,2012,...,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1083971,17,17.9,17.9.1,DC_FTA_TOTAL,Total official development assistance (gross d...,894,Zambia,2013,84.15022,2013,...,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1083972,17,17.9,17.9.1,DC_FTA_TOTAL,Total official development assistance (gross d...,894,Zambia,2014,124.97344,2014,...,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1083973,17,17.9,17.9.1,DC_FTA_TOTAL,Total official development assistance (gross d...,894,Zambia,2015,93.49433,2015,...,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1083974,17,17.9,17.9.1,DC_FTA_TOTAL,Total official development assistance (gross d...,894,Zambia,2016,94.33974,2016,...,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The data set is structured by indicators and years in rows in one large data frame with all countries. We would like to have one data frame per country. Hence, we first extract the names of *regional groupings*, i.e. countries, continents, etc., and the names of so-called *other groupings*.

According to the UN Statistics Division, other groupings include Least Developed Countries (LDC), Land Locked Developing Countries (LLDC), Small Island Developing States (SIDS), Developed Regions, and Developing Regions. 

Developing Regions are Latin America and the Caribbean, South-Eastern Asia, Southern Asia, Southern Asia (excluding India), Caucasus and Central Asia, Eastern Asia (excluding Japan and China), Western Asia (exc. Armenia, Azerbaijan, Cyprus, Israel and Georgia), Eastern Asia (excluding Japan), Oceania (exc. Australia and New Zealand), Sub-Saharan Africa (inc. Sudan), and Northern Africa (exc. Sudan).

**All these groupings can be subject to separate network analyses of the indicators later on.**




Let's first see all different columns of our data frame before we only see these different groupings.

In [4]:
list(all_data)

['Goal',
 'Target',
 'Indicator',
 'SeriesCode',
 'SeriesDescription',
 'GeoAreaCode',
 'GeoAreaName',
 'TimePeriod',
 'Value',
 'Time_Detail',
 'Source',
 'FootNote',
 'Nature',
 'Units',
 '[Age]',
 '[Bounds]',
 '[Cities]',
 '[Education level]',
 '[Freq]',
 '[Hazard type]',
 '[IHR Capacity]',
 '[Level/Status]',
 '[Location]',
 '[Migratory status]',
 '[Mode of transportation]',
 '[Name of international institution]',
 '[Name of non-communicable disease]',
 '[Quantile]',
 '[Reporting Type]',
 '[Sex]',
 '[Tariff regime (status)]',
 '[Type of mobile technology]',
 '[Type of occupation]',
 '[Type of product]',
 '[Type of skill]',
 '[Type of speed]']

We have even lots of information on a sub-indicator level and this might be subject to more detailed analyses later on. We could, e.g., indicator 4.6.1* explore by disparate age goups and by sex.

\* *Indicator 4.6.1: Proportion of population in a given age group achieving at least a fixed level of proficiency in functional (a) literacy and (b) numeracy skills, by sex.*


We keep this possibility open, but now, let's not go further into a sub-indicator level and see the different groupings.

In [5]:
groupings = all_data['GeoAreaName'].unique()
groupings.tolist()

['World',
 'Albania',
 'Algeria',
 'Angola',
 'Azerbaijan',
 'Argentina',
 'Australia',
 'Austria',
 'Bangladesh',
 'Armenia',
 'Belgium',
 'Bhutan',
 'Bolivia (Plurinational State of)',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Belize',
 'Solomon Islands',
 'Bulgaria',
 'Myanmar',
 'Burundi',
 'Belarus',
 'Cameroon',
 'Canada',
 'Cabo Verde',
 'Central African Republic',
 'Sri Lanka',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Democratic Republic of the Congo',
 'Costa Rica',
 'Croatia',
 'Cyprus',
 'Czechia',
 'Benin',
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'El Salvador',
 'Ethiopia',
 'Estonia',
 'Fiji',
 'Finland',
 'France',
 'Djibouti',
 'Gabon',
 'Georgia',
 'Gambia',
 'State of Palestine',
 'Germany',
 'Ghana',
 'Kiribati',
 'Greece',
 'Guatemala',
 'Guinea',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran (Islamic Republic of)',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 "Côte d'Ivoire",
 'Jama

We convert the data set into multiple small data sets by creating a dictionary that contains the groupings' names as keys. 

First, we create empty data frames for each key.

In [6]:
dict_all = {group: pd.DataFrame() for group in groupings}

In [7]:
# check, should be empty
dict_all.get('Belize')

""


Second, we replace each of the empty data frames with the data we have available for them. Note, that our dictionary will be the ensamble of all groupings.

In [8]:
for group in groupings:    # memory-intensive
    dict_all[group] = all_data[all_data['GeoAreaName'].isin(['{}'.format(group)])]

In [9]:
# check
dict_all['Belize']

,Goal,Target,Indicator,SeriesCode,SeriesDescription,GeoAreaCode,GeoAreaName,TimePeriod,Value,Time_Detail,...,[Name of non-communicable disease],[Quantile],[Reporting Type],[Sex],[Tariff regime (status)],[Type of mobile technology],[Type of occupation],[Type of product],[Type of skill],[Type of speed]
158,1,1.1,1.1.1,SI_POV_DAY1,Proportion of population below international p...,84,Belize,1993,10.1,1993,...,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159,1,1.1,1.1.1,SI_POV_DAY1,Proportion of population below international p...,84,Belize,1994,9.9,1994,...,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,1,1.1,1.1.1,SI_POV_DAY1,Proportion of population below international p...,84,Belize,1996,13.3,1996,...,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,1,1.1,1.1.1,SI_POV_DAY1,Proportion of population below international p...,84,Belize,1997,16,1997,...,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
162,1,1.1,1.1.1,SI_POV_DAY1,Proportion of population below international p...,84,Belize,1998,15.1,1998,...,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
163,1,1.1,1.1.1,SI_POV_DAY1,Proportion of population below international p...,84,Belize,1999,14.9,1999,...,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10809,1,1.3,1.3.1,SI_COV_LMKT,[World Bank] Proportion of population covered ...,84,Belize,2009,5.7211,2009,...,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10966,1,1.3,1.3.1,SI_COV_LMKTPQ,[World Bank] Poorest quintile covered by labou...,84,Belize,2009,4.08667,2009,...,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11246,1,1.3,1.3.1,SI_COV_PENSN,[ILO] Proportion of population above statutory...,84,Belize,2011,64.6,NaN,...,NaN,NaN,G,BOTHSEX,NaN,NaN,NaN,NaN,NaN,NaN
11513,1,1.3,1.3.1,SI_COV_SOCAST,[World Bank] Proportion of population covered ...,84,Belize,2009,16.27566,2009,...,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now, we have one data frame per country. The next step is to have years as columns.

The next cell gives us the series codes in the rows and the years in the columns. These series codes are unique descriptions of the sub-indicators and we match these series codes to indicators and all other information in a different data frame.

In [10]:
for group in groupings:
    dict_all[group] = dict_all.get(group).pivot_table(values='Value', index=['SeriesCode'], columns='TimePeriod', dropna=False, aggfunc='first')

In [11]:
# check
dict_all['Guam'].head()

TimePeriod,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
SeriesCode,,,,,,,,,,,,,,,,,,,,,
AG_LND_FRST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.3,NaN,...,NaN,46.3,NaN,NaN,NaN,NaN,46.3,NaN,NaN,NaN
AG_LND_FRSTBIOM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.8,NaN,...,NaN,2.8,NaN,NaN,NaN,NaN,2.8,NaN,NaN,NaN
AG_LND_FRSTBIOPHA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112,NaN,...,NaN,112,NaN,NaN,NaN,NaN,112,NaN,NaN,NaN
AG_LND_FRSTCERT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN
AG_LND_FRSTCHG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN


Let's now save a data frame with all of the meta-information. We delete the columns which are specific in area and time, and of course we do not want to have the values in this data frame. In the end, we delete all duplicate entries in the column **SeriesCode**. So, we are left with the information we wanted: mapping the series codes to the indicators, the Source for the data, the Units measured in, etc.

In [12]:
info = all_data.drop(columns=['GeoAreaCode', 'GeoAreaName', 'TimePeriod', 'Value', 'Time_Detail']).drop_duplicates(subset='SeriesCode')

In [13]:
# check
info.head()

,Goal,Target,Indicator,SeriesCode,SeriesDescription,Source,FootNote,Nature,Units,[Age],...,[Name of non-communicable disease],[Quantile],[Reporting Type],[Sex],[Tariff regime (status)],[Type of mobile technology],[Type of occupation],[Type of product],[Type of skill],[Type of speed]
0,1,1.1,1.1.1,SI_POV_DAY1,Proportion of population below international p...,"World Bank, Development Research Group. Data a...",World aggregate.,G,PERCENT,NaN,...,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1333,1,1.1,1.1.1,SI_POV_EMP1,Employed population below international povert...,"ILO estimates, November 2017, available in ILO...",NaN,M,PERCENT,15-24,...,NaN,NaN,G,BOTHSEX,NaN,NaN,NaN,NaN,NaN,NaN
9703,1,1.2,1.2.1,SI_POV_NAHC,Proportion of population living below the nati...,"Source: World Development Indicators database,...",Source: Central Statistic Organization (CSO) ...,CA,PERCENT,NaN,...,NaN,NaN,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10476,1,1.3,1.3.1,SI_COV_BENFTS,[ILO] Proportion of population covered by at l...,ILO estimates based on country data compled th...,NaN,E,PERCENT,NaN,...,NaN,NaN,G,BOTHSEX,NaN,NaN,NaN,NaN,NaN,NaN
10581,1,1.3,1.3.1,SI_COV_CHLD,[ILO] Proportion of children/households receiv...,ILO estimates based on country data compled th...,NaN,E,PERCENT,NaN,...,NaN,NaN,G,BOTHSEX,NaN,NaN,NaN,NaN,NaN,NaN


## Cleaning up and transforming all country data frames into the same dimensions

We have a couple of things to do to make our data frames workable:
1. We have some values in the data frames which we do not want, as e.g. <code>,</code>, <code> = </code>, <code>N</code>, etc. We replace them with appropriate values, i.e. <code>0</code>, or simply a space. 
2. Some data frames have data from **1990** to **2018**, some others from **1992** to **2018**. We want to have all data frames having data from **1990** to **2018**, i.e. an equal amount of columns. The additional columns are filled with <code>NaNs</code>.
3. Some data frames have not all indicators and sub-indicators listed, but we would like to have all of them in all data frames. These additional rows are filled with <code>NaNs</code>.

Let's start with the first task, i.e. cleaning up the data frames.

We first need to define lists for all years, i.e. **1990** to **2018** and all indicators and sub-indicators, i.e. series codes.

In [15]:
# list of all years
years = list(dict_all['France'])    # France is an example of a country that has all columns
years

['1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018']

Change <span style="color:red"> 'Guam' </span> in the cell below to a few other countries and you'll see that they can have different lengths. We need to bring all on the same length. We agree on having data for the **years 1990 to 2018**.

Now, we insert the missing years for all groupings. We want to have NaNs in those columns.

In [ ]:
# example
list(dict_all['Guam'])

In [16]:
# list of all series codes
seriescodes = list(info['SeriesCode'])
seriescodes

['SI_POV_DAY1',
 'SI_POV_EMP1',
 'SI_POV_NAHC',
 'SI_COV_BENFTS',
 'SI_COV_CHLD',
 'SI_COV_DISAB',
 'SI_COV_LMKT',
 'SI_COV_LMKTPQ',
 'SI_COV_MATNL',
 'SI_COV_PENSN',
 'SI_COV_POOR',
 'SI_COV_SOCAST',
 'SI_COV_SOCASTPQ',
 'SI_COV_SOCINS',
 'SI_COV_SOCINSPQ',
 'SI_COV_UEMP',
 'SI_COV_VULN',
 'SI_COV_WKINJRY',
 'SG_DSR_LEGREG',
 'SG_DSR_LGRGSR',
 'SG_DSR_SILN',
 'SG_DSR_SILS',
 'SG_GOV_LOGV',
 'VC_DSR_AFFCT',
 'VC_DSR_AGLH',
 'VC_DSR_AGLN',
 'VC_DSR_CHLN',
 'VC_DSR_CILN',
 'VC_DSR_DADN',
 'VC_DSR_DAFF',
 'VC_DSR_DDHN',
 'VC_DSR_DYDN',
 'VC_DSR_DYHN',
 'VC_DSR_GDPLS',
 'VC_DSR_HOLH',
 'VC_DSR_HOLN',
 'VC_DSR_IJILN',
 'VC_DSR_LSGP',
 'VC_DSR_MISS',
 'VC_DSR_MMHN',
 'VC_DSR_MORT',
 'VC_DSR_MTMN',
 'VC_DSR_MTMP',
 'VC_DSR_PDAN',
 'VC_DSR_PDLN',
 'VC_DSR_PDYN',
 'SD_XPD_ESED',
 'AG_PRD_FIESMSI',
 'AG_PRD_FIESMSIN',
 'AG_PRD_FIESSI',
 'AG_PRD_FIESSIN',
 'SN_ITK_DEFC',
 'SH_STA_OVRWGT',
 'SH_STA_OVRWGTN',
 'SH_STA_STUNT',
 'SH_STA_STUNTN',
 'SH_STA_WASTE',
 'SH_STA_WASTEN',
 'ER_GRF_ANIMKPT',
 

In [17]:
# count how many we have
len(seriescodes)

359

Firstly, we insert the missing years as columns for all groupings.

In [18]:
for group in groupings:    # memory-intensive
    for year in years:
        if year not in list(dict_all[group]):
            dict_all[group]['{}'.format(year)] = np.nan
    # having the years in order
    dict_all[group] = dict_all[group][['1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']]

In [19]:
# check
dict_all['Guam'].head()

TimePeriod,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
SeriesCode,,,,,,,,,,,,,,,,,,,,,
AG_LND_FRST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,46.3,NaN,NaN,NaN,NaN,46.3,NaN,NaN,NaN
AG_LND_FRSTBIOM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.8,NaN,NaN,NaN,NaN,2.8,NaN,NaN,NaN
AG_LND_FRSTBIOPHA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,112,NaN,NaN,NaN,NaN,112,NaN,NaN,NaN
AG_LND_FRSTCERT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN
AG_LND_FRSTCHG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN


Secondly, we insert the missing series codes as rows.

Let's first see how many rows do we have for <span style="color:red"> Guam </span>?

In [ ]:
len(list(dict_all['Guam'].index))

Let's have all $J = 359$ sub-indicators we want for each country as rows. We fill these rows with NaNs. 

In [20]:
for group in groupings:        # memory-intensive
    for seriescode in seriescodes:
        if seriescode not in list(dict_all[group].index):
            dict_all[group].loc[seriescode] = np.nan    # fill these rows with NaNs

In [21]:
# check: do we have 359?
len(list(dict_all['Guam'].index))

359

In [22]:
# convert all to floats (~10 minutes  computing time)
for group in groupings:
    for year in years:    
        for seriescode in seriescodes:
            if not isinstance(dict_all[group][year][seriescode], float):
                dict_all[group][year][seriescode] = float(dict_all[group][year][seriescode].replace(',', '').replace('<', '').replace('>', '').replace('=', '').replace('N', '0').replace(' -   ', '0'))

/home/felix/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
# double-check: are all series codes as rows?
list(dict_all['Guam'].index)

['AG_LND_FRST',
 'AG_LND_FRSTBIOM',
 'AG_LND_FRSTBIOPHA',
 'AG_LND_FRSTCERT',
 'AG_LND_FRSTCHG',
 'AG_LND_FRSTN',
 'AG_LND_TOTL',
 'EG_ELC_ACCS',
 'EG_FEC_RNEW',
 'EG_TBA_H2CO',
 'EG_TBA_H2COAQ',
 'EG_TBA_H2CORL',
 'EN_WBE_PMNR',
 'EN_WBE_PMPN',
 'EN_WBE_PMPP',
 'EN_WBE_PMPR',
 'ER_GRF_ANIMKPT',
 'ER_GRF_ANIMRCNT',
 'ER_GRF_ANIMRCNTN',
 'ER_GRF_ANIMSTOR',
 'ER_GRF_ANIMSTORN',
 'ER_MRN_MARIN',
 'ER_MRN_MARINT',
 'ER_MRN_MPA',
 'ER_NRK_LBRED',
 'ER_NRK_LBREDN',
 'ER_PTD_TERRS',
 'ER_RSK_LBRED',
 'ER_RSK_LBREDN',
 'ER_RSK_LSTI',
 'ER_UNK_LBRED',
 'ER_UNK_LBREDN',
 'GB_POP_SCIERD',
 'GB_XPD_RSDV',
 'IT_MOB_NTWK',
 'IT_NET_BBN',
 'IT_NET_BBP',
 'IT_USE_ii99',
 'SG_REG_BRTH90N',
 'SG_REG_CENSUSN',
 'SG_REG_DETH75N',
 'SH_SAN_DEFECT',
 'SH_TBS_INCID',
 'SL_EMP_PCAP',
 'SP_DYN_ADKL',
 'VC_DSR_AFFCT',
 'VC_DSR_AGLH',
 'VC_DSR_AGLN',
 'VC_DSR_BSDN',
 'VC_DSR_CDAN',
 'VC_DSR_CDYN',
 'VC_DSR_CHLN',
 'VC_DSR_CILN',
 'VC_DSR_DADN',
 'VC_DSR_DAFF',
 'VC_DSR_DDHN',
 'VC_DSR_DYDN',
 'VC_DSR_DYHN',
 'VC

Finally, we can save all countries as different <code>csv</code> files.

In [24]:
if not os.path.exists('csv'):
    os.mkdir('csv')

In [25]:
for group in groupings:
    dict_all[group].to_csv(r'csv/{}.csv'.format(group))

Having the information file might also be helpful.

In [26]:
info.to_csv(r'info.csv')